In [5]:
import pandas as pd
import numpy as np
import xlsxwriter
from prophet import Prophet
from numpy import concatenate
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from datetime import datetime
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from math import sqrt

# prophet univariate
all_df = pd.read_csv('iztech.csv', usecols=['Name', 'WS101']) # veri seti okundu
all_df.columns = ['ds', 'y'] # kolon isimleri değiştirildi
results = [] # sonuçların tutulacağı liste oluşturuldu

for i in range(0, 57601, 2): # 10 dakikalık verileri 2'şer 2'şer alarak 5 dakikalık veri setleri oluşturuldu
    df = all_df[i:i+144] # 5 dakikalık veri seti oluşturuldu
    m = Prophet(daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=True) # model oluşturuldu
    m.fit(df) # model eğitildi
    future = m.make_future_dataframe(periods=144, freq='10 min') # 5 dakikalık gelecek veriler oluşturuldu
    forecast = m.predict(future) # gelecek veriler tahmin edildi
    result = forecast[['ds', 'yhat']] # tahmin edilen verilerden sadece tarih ve tahmin edilen değerler alındı
    result = result.tail(2) # 5 dakikalık gelecek verilerden sadece 1 veri alındı
    results.append(result) # sonuçlar listesine eklendi

# prophet multivariate
train = pd.read_csv('iztech.csv', usecols=['Name', 'WS101', 'RH3', 'T3', 'P2']) # veri seti okundu
train.rename(columns={'Name': 'ds', 'RH3': 'add1', 'T3': 'add2', 'P2': 'add3', 'WS101': 'y'}, inplace=True) # kolon isimleri değiştirildi
results_multivariate = [] # sonuçların tutulacağı liste oluşturuldu

for i in range(0, 57601, 2): # 10 dakikalık verileri 2'şer 2'şer alarak 5 dakikalık veri setleri oluşturuldu
    X_train = train.iloc[i:i+144] # 5 dakikalık veri seti oluşturuldu
    X_val = train.iloc[i+144:i+146] # 5 dakikalık gelecek veriler oluşturuldu
    model = Prophet(daily_seasonality=True, weekly_seasonality=True, yearly_seasonality=True) # model oluşturuldu
    model.add_regressor('add1') # modelin kullanacağı ek özellikler belirtildi
    model.add_regressor('add2') # modelin kullanacağı ek özellikler belirtildi
    model.add_regressor('add3') # modelin kullanacağı ek özellikler belirtildi
    model.fit(X_train) # model eğitildi
    forecast_1 = model.predict(X_val.drop(columns="y")) # gelecek veriler tahmin edildi
    result_1 = forecast_1[['ds', 'yhat']] # tahmin edilen verilerden sadece tarih ve tahmin edilen değerler alındı
    results_multivariate.append(result_1) # sonuçlar listesine eklendi

# Create Pandas dataframes from forecasts
df1 = pd.read_csv('iztech.csv', usecols=['Name', 'T3']) # veri seti okundu
df1.columns = ['ds', 'y'] # kolon isimleri değiştirildi
df1 = df1[144:57601] # 5 dakikalık gelecek verilerden sadece 1 veri alındı
df1 = pd.DataFrame(df1) # veri seti dataframe'e çevrildi
df2 = pd.DataFrame(np.concatenate(results), columns=['ds', 'yhat']) # sonuçlar listesi dataframe'e çevrildi
df3 = pd.DataFrame(np.concatenate(results_multivariate), columns=['ds', 'yhat']) # sonuçlar listesi dataframe'e çevrildi
expected = df1['y'] # gerçek değerler alındı
prediction_multi = df2['yhat'] # tahmin edilen değerler alındı
prediction_uni = df3['yhat'] # tahmin edilen değerler alındı

MSE_multi = mean_squared_error(expected, prediction_multi) # çok değişkenli modelin hata değerleri hesaplandı
R2_multi = r2_score(expected, prediction_multi) # çok değişkenli modelin hata değerleri hesaplandı
RMSE_multi = sqrt(mean_squared_error(expected, prediction_multi)) # çok değişkenli modelin hata değerleri hesaplandı
MAE_multi = mean_absolute_error(expected, prediction_multi) # çok değişkenli modelin hata değerleri hesaplandı

print("MSE Multi:", MSE_multi) # çok değişkenli modelin hata değerleri ekrana yazdırıldı
print("R2 Multi:", R2_multi) # çok değişkenli modelin hata değerleri ekrana yazdırıldı
print("RMSE Multi:", RMSE_multi)  # çok değişkenli modelin hata değerleri ekrana yazdırıldı 
print("MAE Multi:", MAE_multi) # çok değişkenli modelin hata değerleri ekrana yazdırıldı


ValueError: Usecols do not match columns, columns expected but not found: ['WS101', 'Name']